In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [2]:
# 72 = WWC, 37 = FA WSL, 43 = MWC, 49 = NWSL
competition = 'FA WSL 19-20'
compID = 37
seasonID = 42

In [3]:
match_info_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/"+str(compID)+"/"+str(seasonID)+".json"

# Pull data from url into python dictionary
games = requests.get(url=match_info_url).json()

# Use list comprehension to create a list of match ids
match_ids = [g['match_id'] for g in games]


print("There are {} games in this list".format(len(match_ids)))

There are 88 games in this list


In [4]:
MatchData = [] 

for p in games:
    attributes = {
        "Home": p['home_team']['home_team_name'],
        "Away": p['away_team']['away_team_name'],
        "Competition": p['competition']['competition_name'],
        "Home Score": p['home_score'],
        "Away Score": p['away_score'],
        "Date": p['match_date'],
        "Match ID": p['match_id']

    }

    MatchData.append(attributes)
    
Matches = pd.DataFrame(data=MatchData, columns=['Competition', 'Date', 'Match ID','Home', 'Home Score', 'Away', 'Away Score'])
Matches.head(10)

,Competition,Date,Match ID,Home,Home Score,Away,Away Score
0,FA Women's Super League,2020-01-05,2275054,Brighton & Hove Albion WFC,1,Liverpool WFC,0
1,FA Women's Super League,2020-01-05,2275072,Chelsea FCW,3,Reading WFC,1
2,FA Women's Super League,2020-01-05,2275085,Tottenham Hotspur Women,1,Manchester City WFC,4
3,FA Women's Super League,2020-01-19,2275113,West Ham United LFC,2,Brighton & Hove Albion WFC,1
4,FA Women's Super League,2019-03-14,19800,Arsenal WFC,4,Bristol City WFC,0
5,FA Women's Super League,2020-01-05,2275142,Manchester United,0,Bristol City WFC,1
6,FA Women's Super League,2019-09-08,2275099,Liverpool WFC,0,Reading WFC,1
7,FA Women's Super League,2020-02-02,2275057,Chelsea FCW,8,West Ham United LFC,0
8,FA Women's Super League,2019-12-15,2275075,Reading WFC,3,Tottenham Hotspur Women,1
9,FA Women's Super League,2020-01-19,2275092,Birmingham City WFC,0,Manchester City WFC,2


In [5]:
Matches.to_csv('/Users/michael/Documents/Python/CSV/'+str(competition)+' Matches.csv')

In [6]:
match_ids = []
for g in games:
    m_id = g['match_id']
    match_ids.append(m_id)

In [7]:
PassData = []

# template URL at which each game's event data is stored
match_data_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{}.json"

# This takes some time, naturally
for match_id in match_ids:
    
    # Get all events from a particular game
    all_events = requests.get(url=match_data_url.format(match_id)).json()

    # Filter them by type-name Pass
    passes = [x for x in all_events if x['type']['name'] == "Pass"]
    
    for p in passes:
        attributes = {
            "Player": p['player']['name'],
            "Team": p['team']['name'],
            "X": p['location'][0],
            "Y": p['location'][1],
            "DestX": p['pass']['end_location'][0],
            "DestY": p['pass']['end_location'][1],
            "Length": p['pass']['length'],
            "Angle": p['pass']['angle'],
            # Multiple other features can be collected here, such as pressure
            "ifComplete": 0 if "outcome" in p['pass'] else 1,
            "PassesAttempted": 1 if "outcome" in p['pass'] else 1,
            "Height": p['pass']['height']['name'],
            "Position": p['position']['name'],
            "Type of Attack": p['play_pattern']['name'],
            "Minute": p['minute'],
            "Second": p['second'],
            "Possession": p['possession'],
            "Type": p['type']['name'],
            "Half": p['period'],
            "Recipient": p['pass']['recipient']['name'] if "recipient" in p['pass'] else "None",
            "UnderPressure": 1 if "under_pressure" in p else 0,
            "ifAssist": 1 if "goal_assist" in p['pass'] else 0,
            "ThroughBall": 1 if "through_ball" in p['pass'] else 0,
            "MatchID": match_id,
            "CompetitionID": compID
        }
        
        PassData.append(attributes)

In [8]:
PassData = pd.DataFrame(data=PassData, columns=['CompetitionID','MatchID','Team', 'Player', 'Position', 'ifComplete', 'ifAssist', 'X', 'Y', 'DestX', 'DestY', 'Height', 'Angle', 'Length', 'Type of Attack', 'Possession', 'PassesAttempted', 'Half','Recipient', "ThroughBall", 'UnderPressure', 'Minute', 'Second'])
PassData.head()

,CompetitionID,MatchID,Team,Player,Position,ifComplete,ifAssist,X,Y,DestX,...,Length,Type of Attack,Possession,PassesAttempted,Half,Recipient,ThroughBall,UnderPressure,Minute,Second
0,37,2275054,Brighton & Hove Albion WFC,Léa Le Garrec,Left Center Midfield,1,0,61.0,40.1,37.0,...,24.100622,From Kick Off,2,1,1,Maya Le Tissier,0,0,0,0
1,37,2275054,Brighton & Hove Albion WFC,Maya Le Tissier,Left Center Back,1,0,36.2,39.7,29.6,...,17.771044,From Kick Off,2,1,1,Danique Kerkdijk,0,0,0,2
2,37,2275054,Brighton & Hove Albion WFC,Danique Kerkdijk,Right Center Back,1,0,21.4,58.8,19.5,...,24.075090,From Kick Off,2,1,1,Maya Le Tissier,0,0,0,6
3,37,2275054,Brighton & Hove Albion WFC,Maya Le Tissier,Left Center Back,0,0,21.2,34.2,65.5,...,60.702060,From Kick Off,2,1,1,Kayleigh Green,0,0,0,9
4,37,2275054,Liverpool WFC,Leigh-Anne Robe,Left Back,0,0,54.6,4.4,71.1,...,17.051100,From Kick Off,2,1,1,Niamh Charles,0,0,0,13


In [9]:
PassData['ifGround'] = [1 if x == 'Ground Pass' else 0 for x in PassData['Height']]
PassData['ifLow'] = [1 if x == 'Low Pass' else 0 for x in PassData['Height']]
PassData['ifHigh'] = [1 if x == 'High Pass' else 0 for x in PassData['Height']]
PassData['DistX'] = PassData['DestX'] - PassData['X']
PassData['PP'] = [1 if x >= 23 else 0 for x in PassData['DistX']]

In [10]:
PassData['IntoF3'] = [1 if x >= 80 else 0 for x in PassData['DestX']]
PassData['From23'] = [1 if x <= 79 else 0 for x in PassData['X']]
PassData['DPSum'] = PassData['IntoF3'] + PassData['From23']
PassData['DP'] = [1 if x == 2 else 0 for x in PassData['DPSum']]

In [11]:
PassData = pd.DataFrame(data=PassData, columns=['CompetitionID','MatchID','Team', 'Player', 'Position', 'ifComplete', 'ifAssist', 'X', 'Y', 'DestX', 'DestY', 'Height', 'Angle', 'Length', 'Type of Attack', 'Possession', 'Minute', 'Second', 'PassesAttempted', 'Half','Recipient', 'ThroughBall', 'UnderPressure', 'PP', 'DP', 'ifGround', 'ifLow', 'ifHigh'])
PassData.head()

,CompetitionID,MatchID,Team,Player,Position,ifComplete,ifAssist,X,Y,DestX,...,PassesAttempted,Half,Recipient,ThroughBall,UnderPressure,PP,DP,ifGround,ifLow,ifHigh
0,37,2275054,Brighton & Hove Albion WFC,Léa Le Garrec,Left Center Midfield,1,0,61.0,40.1,37.0,...,1,1,Maya Le Tissier,0,0,0,0,1,0,0
1,37,2275054,Brighton & Hove Albion WFC,Maya Le Tissier,Left Center Back,1,0,36.2,39.7,29.6,...,1,1,Danique Kerkdijk,0,0,0,0,1,0,0
2,37,2275054,Brighton & Hove Albion WFC,Danique Kerkdijk,Right Center Back,1,0,21.4,58.8,19.5,...,1,1,Maya Le Tissier,0,0,0,0,1,0,0
3,37,2275054,Brighton & Hove Albion WFC,Maya Le Tissier,Left Center Back,0,0,21.2,34.2,65.5,...,1,1,Kayleigh Green,0,0,1,0,0,0,1
4,37,2275054,Liverpool WFC,Leigh-Anne Robe,Left Back,0,0,54.6,4.4,71.1,...,1,1,Niamh Charles,0,0,0,0,0,0,1


In [12]:
PassData.to_csv('/Users/michael/Documents/Python/CSV/'+str(competition)+' Passes.csv')

In [13]:
DribbleData = []

# template URL at which each game's event data is stored
match_data_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{}.json"

# This takes some time, naturally
for match_id in match_ids:
    
    # Get all events from a particular game
    all_events = requests.get(url=match_data_url.format(match_id)).json()
        
    # Filter them by type-name Pass
    dribble = [x for x in all_events if x['type']['name'] == "Dribble"]
    
    for p in dribble:
        attributes = {
            "Player": p['player']['name'],
            "Team": p['team']['name'],
            "X": p['location'][0],
            "Y": p['location'][1],
            # Multiple other features can be collected here, such as pressure
            "Outcome":p['dribble']['outcome']['name'],
            "Position": p['position']['name'],
            "Type of Attack": p['play_pattern']['name'],
            "Minute": p['minute'],
            "Second": p['second'],
            "Possession": p['possession'],
            "Type": p['type']['name'],
            "Half": p['period'],
            "UnderPressure": p['under_pressure'],
            "MatchID": match_id,
            "CompetitionID": compID
        }
        
        DribbleData.append(attributes)

In [14]:
DribbleData = pd.DataFrame(data=DribbleData, columns=['CompetitionID','MatchID','Team', 'Player', 'Position', 'X', 'Y', 'Type of Attack', 'Outcome', 'Possession', 'Minute', 'Second', 'Half'])
DribbleData.head()

,CompetitionID,MatchID,Team,Player,Position,X,Y,Type of Attack,Outcome,Possession,Minute,Second,Half
0,37,2275041,Everton LFC,Simone Magill,Center Forward,108.3,42.8,Regular Play,Complete,20,8,31,1
1,37,2275041,Bristol City WFC,Gemma Evans,Left Back,48.9,2.3,From Goal Kick,Incomplete,22,9,31,1
2,37,2275041,Bristol City WFC,Abigail Harrison,Left Wing,91.0,16.6,From Goal Kick,Complete,22,9,56,1
3,37,2275041,Everton LFC,Megan Finnigan,Right Center Back,23.3,61.0,From Throw In,Complete,36,17,45,1
4,37,2275041,Everton LFC,Chloe Kelly,Left Wing,104.2,14.1,From Throw In,Complete,53,24,40,1


In [15]:
DribbleData.to_csv('/Users/michael/Documents/Python/CSV/'+str(competition)+' Dribbles.csv')

In [16]:
ShotData = []

# template URL at which each game's event data is stored
match_data_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{}.json"

# This takes some time, naturally
for match_id in match_ids:
    
    # Get all events from a particular game
    all_events = requests.get(url=match_data_url.format(match_id)).json()
        
    # Filter them by type-name Pass
    shots = [x for x in all_events if x['type']['name'] == "Shot"]
    
    for s in shots:
        attributes = {
            "Player": s['player']['name'],
            "Team": s['team']['name'],
            "X": s['location'][0],
            "Y": s['location'][1],
            "xG": s['shot']['statsbomb_xg'],
            "Position": s['position']['name'],
            "Possession": s['possession'],
            "Type of Attack": s['play_pattern']['name'],
            "Body Part": s['shot']['body_part']['name'],
            "Action": s['shot']['outcome']['name'] if "outcome" in s['shot'] else "Off T",
            "Technique":s['shot']['technique']['name'],
            "OPorFK": s['shot']['type']['name'] if "type" in s['shot'] else 0,
            "Half": s['period'],
            "Minute": s['minute'],
            "Second": s['second'],
            "MatchID": match_id,
            "CompetitionID": compID
            # Multiple other features can be collected here, such as pressure
           
        }
        
        ShotData.append(attributes)

In [17]:
ShotData = pd.DataFrame(data=ShotData, columns=['CompetitionID','MatchID', 'Possession','Team', 'Player', 'Position', 'xG', 'Action', 'Type of Attack', 'OPorFK','Body Part', 'X', 'Y', 'Technique', 'Half', 'Minute', 'Second'])
ShotData.head()

,CompetitionID,MatchID,Possession,Team,Player,Position,xG,Action,Type of Attack,OPorFK,Body Part,X,Y,Technique,Half,Minute,Second
0,37,2275041,20,Everton LFC,Simone Magill,Center Forward,0.082348,Saved,Regular Play,Open Play,Right Foot,111.1,24.0,Normal,1,8,24
1,37,2275041,20,Everton LFC,Simone Magill,Center Forward,0.128042,Blocked,Regular Play,Open Play,Right Foot,109.7,41.4,Normal,1,8,31
2,37,2275041,40,Bristol City WFC,Yana Daniels,Center Attacking Midfield,0.092300,Off T,From Throw In,Open Play,Right Foot,102.6,41.9,Half Volley,1,19,34
3,37,2275041,42,Bristol City WFC,Yana Daniels,Center Attacking Midfield,0.101910,Saved,Regular Play,Open Play,Right Foot,108.2,52.2,Normal,1,20,8
4,37,2275041,49,Everton LFC,Gabrielle George,Left Center Back,0.139322,Off T,From Corner,Open Play,Head,114.3,35.2,Normal,1,23,9


In [18]:
ShotData['ifGoal'] = [1 if x == 'Goal' else 0 for x in ShotData['Action']]
ShotData['ifPen'] = [1 if x == 'Penalty' else 0 for x in ShotData['OPorFK']]
ShotData['Shots'] = [1 if x >= 0 else 0 for x in ShotData['xG']]
ShotData.head()

,CompetitionID,MatchID,Possession,Team,Player,Position,xG,Action,Type of Attack,OPorFK,Body Part,X,Y,Technique,Half,Minute,Second,ifGoal,ifPen,Shots
0,37,2275041,20,Everton LFC,Simone Magill,Center Forward,0.082348,Saved,Regular Play,Open Play,Right Foot,111.1,24.0,Normal,1,8,24,0,0,1
1,37,2275041,20,Everton LFC,Simone Magill,Center Forward,0.128042,Blocked,Regular Play,Open Play,Right Foot,109.7,41.4,Normal,1,8,31,0,0,1
2,37,2275041,40,Bristol City WFC,Yana Daniels,Center Attacking Midfield,0.092300,Off T,From Throw In,Open Play,Right Foot,102.6,41.9,Half Volley,1,19,34,0,0,1
3,37,2275041,42,Bristol City WFC,Yana Daniels,Center Attacking Midfield,0.101910,Saved,Regular Play,Open Play,Right Foot,108.2,52.2,Normal,1,20,8,0,0,1
4,37,2275041,49,Everton LFC,Gabrielle George,Left Center Back,0.139322,Off T,From Corner,Open Play,Head,114.3,35.2,Normal,1,23,9,0,0,1


In [19]:
OGFData = []

# template URL at which each game's event data is stored
match_data_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{}.json"

# This takes some time, naturally
for match_id in match_ids:
    
    # Get all events from a particular game
    all_events = requests.get(url=match_data_url.format(match_id)).json()
        
    # Filter them by type-name Pass
    OGF = [x for x in all_events if x['type']['name'] == "Own Goal For"]
    
    
    for s in OGF:
        attributes = {
            "Team": s['team']['name'],
            "Action": s['type']['name'],
            "Half": s['period'],
            "Minute": s['minute'],
            "Second": s['second'],
            "MatchID": match_id,
            "Possession": s['possession'],
            "CompetitionID": compID
            # Multiple other features can be collected here, such as pressure
        }
        
        OGFData.append(attributes)

In [20]:
OGFData = pd.DataFrame(data=OGFData, columns=['CompetitionID','MatchID', 'Possession', 'Team', 'Player', 'Position', 'xG', 'Action', 'Type of Attack', 'OPorFK','Body Part', 'X', 'Y', 'Technique', 'Half', 'Minute', 'Second',])
OGFData.head()

,CompetitionID,MatchID,Possession,Team,Player,Position,xG,Action,Type of Attack,OPorFK,Body Part,X,Y,Technique,Half,Minute,Second
0,37,2275120,116,Everton LFC,NaN,NaN,NaN,Own Goal For,NaN,NaN,NaN,NaN,NaN,NaN,2,45,30
1,37,2275139,134,Manchester United,NaN,NaN,NaN,Own Goal For,NaN,NaN,NaN,NaN,NaN,NaN,2,51,8
2,37,2275126,52,Everton LFC,NaN,NaN,NaN,Own Goal For,NaN,NaN,NaN,NaN,NaN,NaN,1,25,2


In [21]:
OGFData['ifGoal'] = [1 if x == 'Own Goal For' else 0 for x in OGFData['Action']]
OGFData['Shots'] = [0 if x == 'Own Goal For' else 0 for x in OGFData['Action']]
OGFData = OGFData.replace(to_replace = np.nan, value =0) 

In [22]:
frames = [ShotData, OGFData]
ShotData = pd.concat(frames)
ShotData = pd.DataFrame(data=ShotData, columns=['CompetitionID','MatchID', 'Possession', 'Team', 'Player', 'Position', 'Action', 'xG', 'ifGoal', 'Shots', 'ifPen', 'Type of Attack', 'OPorFK','Body Part', 'X', 'Y', 'Technique', 'Half', 'Minute', 'Second'])
ShotData.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,CompetitionID,MatchID,Possession,Team,Player,Position,Action,xG,ifGoal,Shots,ifPen,Type of Attack,OPorFK,Body Part,X,Y,Technique,Half,Minute,Second
0,37,2275041,20,Everton LFC,Simone Magill,Center Forward,Saved,0.082348,0,1,0.0,Regular Play,Open Play,Right Foot,111.1,24.0,Normal,1,8,24
1,37,2275041,20,Everton LFC,Simone Magill,Center Forward,Blocked,0.128042,0,1,0.0,Regular Play,Open Play,Right Foot,109.7,41.4,Normal,1,8,31
2,37,2275041,40,Bristol City WFC,Yana Daniels,Center Attacking Midfield,Off T,0.092300,0,1,0.0,From Throw In,Open Play,Right Foot,102.6,41.9,Half Volley,1,19,34
3,37,2275041,42,Bristol City WFC,Yana Daniels,Center Attacking Midfield,Saved,0.101910,0,1,0.0,Regular Play,Open Play,Right Foot,108.2,52.2,Normal,1,20,8
4,37,2275041,49,Everton LFC,Gabrielle George,Left Center Back,Off T,0.139322,0,1,0.0,From Corner,Open Play,Head,114.3,35.2,Normal,1,23,9


In [23]:
ShotData.to_csv('/Users/michael/Documents/Python/CSV/'+str(competition)+' Shots.csv')

In [24]:
PressureData = []

# template URL at which each game's event data is stored
match_data_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{}.json"

# This takes some time, naturally
for match_id in match_ids:
    
    # Get all events from a particular game
    all_events = requests.get(url=match_data_url.format(match_id)).json()
        
    # Filter them by type-name Pass
    pressures = [x for x in all_events if x['type']['name'] == "Pressure"]
    
    for p in pressures:
        attributes = {
            "Player": p['player']['name'],
            "Team": p['team']['name'],
            "Position": p['position']['name'],
            "X": p['location'][0],
            "Y": p['location'][1],
            "Type of Attack": p['play_pattern']['name'],
            "Half": p['period'],
            "Minute": p['minute'],
            "Second": p['second'],
            "Duration": p['duration'] if "duration" in p else 0,
            "MatchID": match_id,
            "CompetitionID": compID
        }
        
        PressureData.append(attributes)

In [25]:
PressureData = pd.DataFrame(data=PressureData, columns=['CompetitionID','MatchID','Team', 'Player', 'Position', 'Type of Attack', 'X', 'Y', 'Half', 'Minute','Second','Duration'])
PressureData.head()

,CompetitionID,MatchID,Team,Player,Position,Type of Attack,X,Y,Half,Minute,Second,Duration
0,37,2275041,Everton LFC,Maéva Clemaron,Center Defensive Midfield,From Kick Off,69.3,46.6,1,0,3,0.599995
1,37,2275041,Everton LFC,Maéva Clemaron,Center Defensive Midfield,Regular Play,66.1,44.9,1,0,10,0.969899
2,37,2275041,Bristol City WFC,Georgia Wilson,Right Center Midfield,Regular Play,59.4,33.7,1,0,12,0.537012
3,37,2275041,Bristol City WFC,Jasmine Matthews,Left Center Back,Regular Play,45.7,36.3,1,0,15,0.896052
4,37,2275041,Bristol City WFC,Gemma Evans,Left Back,Regular Play,47.6,14.7,1,0,22,1.055700


In [26]:
PressureData['StartSeconds'] = PressureData['Minute']*60 + PressureData['Second'] - 5
PressureData['EndSeconds'] = (PressureData['Minute']*60 + PressureData['Second']) + PressureData['Duration'] + 15

In [27]:
PressureData = pd.DataFrame(data=PressureData, columns=['CompetitionID','MatchID','Team', 'Player', 'Position', 'Type of Attack', 'X', 'Y', 'Half', 'StartSeconds', 'EndSeconds'])
PressureData.head()

,CompetitionID,MatchID,Team,Player,Position,Type of Attack,X,Y,Half,StartSeconds,EndSeconds
0,37,2275041,Everton LFC,Maéva Clemaron,Center Defensive Midfield,From Kick Off,69.3,46.6,1,-2,18.599995
1,37,2275041,Everton LFC,Maéva Clemaron,Center Defensive Midfield,Regular Play,66.1,44.9,1,5,25.969899
2,37,2275041,Bristol City WFC,Georgia Wilson,Right Center Midfield,Regular Play,59.4,33.7,1,7,27.537012
3,37,2275041,Bristol City WFC,Jasmine Matthews,Left Center Back,Regular Play,45.7,36.3,1,10,30.896052
4,37,2275041,Bristol City WFC,Gemma Evans,Left Back,Regular Play,47.6,14.7,1,17,38.055700


In [28]:
PressureData.to_csv('/Users/michael/Documents/Python/CSV/'+str(competition)+' Pressure.csv')

In [29]:
DuelData = []

# template URL at which each game's event data is stored
match_data_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{}.json"

# This takes some time, naturally
for match_id in match_ids:
    
    # Get all events from a particular game
    all_events = requests.get(url=match_data_url.format(match_id)).json()
        
    # Filter them by type-name Pass
    pressures = [x for x in all_events if x['type']['name'] == "Duel"]
    
    for p in pressures:
        attributes = {
            "Player": p['player']['name'],
            "Team": p['team']['name'],
            "Position": p['position']['name'],
            "X": p['location'][0],
            "Y": p['location'][1],
            "Type of Attack": p['play_pattern']['name'],
            "Half": p['period'],
            "MatchID": match_id,
            "CompetitionID": compID
        }
        
        DuelData.append(attributes)

In [30]:
DuelData = pd.DataFrame(data=DuelData, columns=['CompetitionID','MatchID','Team', 'Player', 'Position', 'Type of Attack', 'X', 'Y'])
DuelData.head()
DuelData.to_csv('/Users/michael/Documents/Python/CSV/'+str(competition)+' Duels.csv')

In [31]:
RecovData = []

# template URL at which each game's event data is stored
match_data_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{}.json"

# This takes some time, naturally
for match_id in match_ids:
    
    # Get all events from a particular game
    all_events = requests.get(url=match_data_url.format(match_id)).json()
        
    # Filter them by type-name Pass
    pressures = [x for x in all_events if x['type']['name'] == "Ball Recovery"]
    
    for p in pressures:
        attributes = {
            "Player": p['player']['name'],
            "Team": p['team']['name'],
            "Position": p['position']['name'],
            "X": p['location'][0],
            "Y": p['location'][1],
            "Type of Attack": p['play_pattern']['name'],
            "Half": p['period'],
            "MatchID": match_id,
            "CompetitionID": compID
        }
        
        RecovData.append(attributes)

In [32]:
RecovData = pd.DataFrame(data=RecovData, columns=['CompetitionID','MatchID','Team', 'Player', 'Position', 'Type of Attack', 'X', 'Y'])
RecovData.head()
RecovData.to_csv('/Users/michael/Documents/Python/CSV/'+str(competition)+' Recoveries.csv')